<h1>An overview of <code>AMRVW</code></h1>

<p>The <code>AMRVW</code> package implements some numerical linear algebra algorithms of Jared L. Aurentz, Thomas Mach, Leonardo Robol, Raf Vandebril, and David S. Watkins for finding eigenvalues of matrices through Francis's method.</p>

<p>An inspiration is to find  the roots of a polynomial through the eigenvalues of a companion matrix. This is implemented in <code>AMRVW</code> through the <code>roots</code> function.</p>

<p>To illustrate,</p>

In [1]:
using AMRVW
const A = AMRVW  # currently there are no exports
ps = [24.0, -50.0, 35.0, -10.0, 1.0]  #  (x-1)(x-2)(x-3)(x-4) = 24 -50x + 25x^2  -10x^3 +  x^4
A.roots(ps)

4-element Array{Complex{Float64},1}:
 0.9999999999999996 + 0.0im
 2.0000000000000027 + 0.0im
 2.9999999999999876 + 0.0im
  4.000000000000012 + 0.0im

<p>The example shows</p>

<ul>
<li><p>the interface expects polynomials specified through a vector of coefficients, <code>&#91;a0, a1, ..., an&#93;</code></p>
</li>
<li><p>the  4 roots, always as complex numbers</p>
</li>
<li><p>the fact that the roots are numeric approximations due to accumulated round off errors.</p>
</li>
</ul>

<p>Similarly, roots of polynomials over the  complex numbers can be found</p>

In [1]:
ps  =  [0.0 + 1.0im, -1.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 - 1.0im, 1.0 + 0.0im] # (x-1)(x+1)(x-i)^2)(x+i)
A.roots(ps)

5-element Array{Complex{Float64},1}:
                   -1.0 + 2.220446049250313e-16im
                   -0.0 + 1.0000000149011605im   
                    0.0 + 0.9999999850988397im   
 1.5265566588595902e-16 - 1.0im                  
     1.0000000000000004 - 5.551115123125785e-17im

<p>There are other ways  to numerically find roots  of polynomials  in <code>Julia</code>, notably the <code>roots</code> function  of  the <code>Polynomials</code> package and the <code>roots</code> function of the <code>PolynomialRoots</code> package:</p>

<ul>
<li><p>unlike <code>Polynomials.roots</code> (but similar to <code>PolynomialRoots.roots</code>) this  <code>roots</code> function can work with  big floats and other floating point types.</p>
</li>
<li><p>For moderate sized polynomials ($n \approx 50$), <code>PolynomialRoots.roots</code> is faster than <code>Polynomials.roots</code> which is faster than <code>roots</code>, though all are fast. When $n \approx 75$, <code>roots</code> is faster (much so for large $n$ than <code>Polynomials.roots</code>).</p>
</li>
<li><p>Unlike both <code>Polynomials.roots</code> and <code>PolynomialRoots.roots</code> this <code>roots</code> function can accurately identify roots of polynomials of high degree.  (For a polynomial with $n$ random coefficients, e.g., <code>ps &#61; rand&#40;n&#41;</code>) <code>Polynomials.roots</code> will have troubles for n around 50; <code>PolynomialRoots.roots</code> will have issues for <code>n</code> around 300; <code>roots</code> can quickly handle degree 3000, and still be accurate for higher degrees.)</p>
</li>
<li><p>The <code>roots</code> function is shown by  the authors  to be  backward stable. The same isn't the case  for the other  two.</p>
</li>
<li><p>In the first example, the residual errors are  similar in size to <code>Polynomials.roots</code>, but  <code>PolynomialRoots.roots</code> the residual errors seem to be generally a bit smaller.</p>
</li>
</ul>

<h2>The companion matrix</h2>

<p>Both <code>roots</code> and <code>Polynomials.roots</code> use a companion matrix representation using the  eigenvalues of this matrix to identify the roots of the polynomial. (The  <code>PolynomialRoots.roots</code> function relies on a different method following a paper by <a href="https://arxiv.org/abs/1203.1034">Skowron and Gould</a>.</p>

<p>Using some functions within <code>AMRVW</code> we can see the companion matrix:</p>

In [1]:
ps = [24.0, -50.0, 35.0, -10.0, 1.0]  #  (x-1)(x-2)(x-3)(x-4) = 24 -50x + 25x^2  -10x^3 +  x^4
state = A.amrvw(ps)
F = Matrix(state) |> round2   # round2 is just M -> round.(M, digits=2)

4×4 Array{Float64,2}:
  0.0   0.0   0.0  24.0
 -1.0   0.0   0.0  50.0
  0.0  -1.0   0.0  35.0
  0.0   0.0  -1.0  10.0

In [1]:
using LinearAlgebra
eigvals(F)

4-element Array{Float64,1}:
 1.0               
 1.9999999999999971
 3.000000000000017 
 3.999999999999985 

<p>The eigvenvalues of this matrix indeed are the roots of the polynomials. Internally, <code>amrvw</code> actually uses an enlarged matrix, with an extra dimension that is not shown here.</p>

<h2>Francis's Algorithm</h2>

<p>Francis's Algorithm begins with a QR decomposition <code>F</code>. For example,</p>

In [1]:
LinearAlgebra.qr(F)

LinearAlgebra.QRCompactWY{Float64,Array{Float64,2}}
Q factor:
4×4 LinearAlgebra.QRCompactWYQ{Float64,Array{Float64,2}}:
 0.0  0.0  0.0  1.0
 1.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0
 0.0  0.0  1.0  0.0
R factor:
4×4 Array{Float64,2}:
 -1.0   0.0   0.0  50.0
  0.0  -1.0   0.0  35.0
  0.0   0.0  -1.0  10.0
  0.0   0.0   0.0  24.0

<p>the decomposition in <code>AMRVW</code> is slightly different, though similar</p>

In [1]:
Matrix(state.QF)

4×4 Array{Float64,2}:
  0.0   0.0   0.0  1.0
 -1.0   0.0   0.0  0.0
  0.0  -1.0   0.0  0.0
  0.0   0.0  -1.0  0.0

In [1]:
Matrix(state.RF) |> round2

5×5 Array{Float64,2}:
 1.0  0.0  -0.0  -50.0   0.0
 0.0  1.0  -0.0  -35.0   0.0
 0.0  0.0   1.0  -10.0   0.0
 0.0  0.0   0.0   24.0  -1.0
 0.0  0.0   0.0    0.0   0.0

<p>(Here the <code>RF</code> matrix shows the extra size used internally in the algorithm.)</p>

<p>The idea of Francis's shifted algorithm is to identify shifts $\rho_1$, $\rho_2$, $\dots$, $\rho_m$ and generate a <em>unitary</em> matrix $V_0 = \alpha (A-\rho_1 I)(A-\rho_2 I)\cdots(A-\rho_m)I \cdot e_1$, $e_1$ being a unit vector with $1$ in the $1$ entry and $0$ elsewhere. As $V_0$ is unitary, the product $V_0^{-1}F V_0$ will have the same eigenvalues.  When $F$ is upper Hessenberg (upper triangular starting with the subdiagonal), as is the case with the companion matrix, then this product will be almost upper Hessenberg, save for a bulge.</p>

<p>In the real-coefficient case,  $m=2$ is used to allow the calculations to be done over the real numbers. For the complex-coefficient case, $m=1$ is possible.</p>

In [1]:
ps  =  [0.0 + 1.0im, -1.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 - 1.0im, 1.0 + 0.0im]
state = A.amrvw(ps)
F = Matrix(state); n = size(F)[1]
M = diagm(0 => ones(Complex{Float64}, 5)) # identity matrix
storage, ctr, m = A.make_storage(state), A.make_counter(state), 1
A.create_bulge(state.QF, state.RF, storage, ctr) # finds shifts and creates V_0
(V0 = storage.VU[1] * M) |> round2

5×5 Array{Complex{Float64},2}:
  0.7+0.07im  -0.71+0.0im   0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.71+0.0im     0.7-0.07im  0.0+0.0im  0.0+0.0im  0.0+0.0im
  0.0+0.0im     0.0+0.0im   1.0+0.0im  0.0+0.0im  0.0+0.0im
  0.0+0.0im     0.0+0.0im   0.0+0.0im  1.0+0.0im  0.0+0.0im
  0.0+0.0im     0.0+0.0im   0.0+0.0im  0.0+0.0im  1.0+0.0im

<p>Up to rounding, $V_0$ is unitary:</p>

In [1]:
isapprox(V0 * V0', M, atol=1e-8)

true

<p>The matrix $V_0' F V_0$ has a bulge below the subdiagonal (the <code>&#91;3,1&#93;</code> position, illustrated with a <code>1</code> below):</p>

In [1]:
V0' * F * V0 |> round2 .|> !iszero

5×5 BitArray{2}:
 1  1  0  0  1
 1  1  0  0  1
 1  1  0  0  0
 0  0  1  0  0
 0  0  0  1  1

<p>The algorithm finds $V_1$ to chase the bulge downward:</p>

In [1]:
A.absorb_Ut(state.QF, state.RF, storage, ctr)
A.passthrough_triu(state.QF, state.RF, storage, ctr, Val(:right))
A.passthrough_Q(state.QF, state.RF, storage, ctr, Val(:right))
(V1 = storage.VU[1] * M) |> round2

5×5 Array{Complex{Float64},2}:
 1.0+0.0im   0.0+0.0im     0.0+0.0im   0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.57+0.11im  -0.82+0.0im   0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.82+0.0im    0.57-0.11im  0.0+0.0im  0.0+0.0im
 0.0+0.0im   0.0+0.0im     0.0+0.0im   1.0+0.0im  0.0+0.0im
 0.0+0.0im   0.0+0.0im     0.0+0.0im   0.0+0.0im  1.0+0.0im

<p>And this produce will have a bulge in <code>&#91;4,2&#93;</code> position:</p>

In [1]:
V1' * (V0' * F * V0) * V1 |> round2 .|> !iszero

5×5 BitArray{2}:
 1  1  1  0  1
 1  1  1  0  1
 0  1  1  0  1
 0  1  1  0  0
 0  0  0  1  1

<p>And again:</p>

In [1]:
A.passthrough_triu(state.QF, state.RF, storage, ctr, Val(:right))
A.passthrough_Q(state.QF, state.RF, storage, ctr, Val(:right))
(V2 = storage.VU[1] * M) |> round2

5×5 Array{Complex{Float64},2}:
 1.0+0.0im  0.0+0.0im   0.0+0.0im     0.0+0.0im   0.0+0.0im
 0.0+0.0im  1.0+0.0im   0.0+0.0im     0.0+0.0im   0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.48+0.14im  -0.87+0.0im   0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.87+0.0im    0.48-0.14im  0.0+0.0im
 0.0+0.0im  0.0+0.0im   0.0+0.0im     0.0+0.0im   1.0+0.0im

In [1]:
V2' * (V1' * (V0' * F * V0) * V1) * V2 |> round2 .|> !iszero

5×5 BitArray{2}:
 1  1  1  1  1
 1  1  1  1  1
 0  1  1  1  1
 0  0  1  1  1
 0  0  1  1  1

<p>Once pushed to the bottom, the bulge is absorbed into the matrix, leaving an upper Hessenberg form.</p>

<p>If the shifts are appropriately chosen, after a few iterations this resulting matrix can have an eigenvalue immediately read off and after deflation subsequent eigenvalues can be found.</p>

<h3>Shifts</h3>

<p>Above the bulge is created with a single rotator. As mentioned, for the real variable case, two rotators are used, so that the computations can be kept using real numbers. In general, the AMRVW algorithm can be defined for $m$ rotators. These rotators are produced by <code>create_bulge</code>, as illustrated above, and stored in <code>storage.UV</code>.</p>

<p>The choice of shifts is <em>essentially</em> the eigenvalues of the lower $2 \times 2$ submatrix (In the $m$-shift case, the lower $m\times m$ submatrix). In the Vandrebril and Watkins paper it is mentioned that this choice <em>normally</em> yields quaratic convergence. That is, one of the rotators will become a diagonal rotator with <code>s</code> part $0$. When that happens, deflation can occur. The algorithm is applied on this smaller, deflated matrix, until the deflated matrix is comprised of no more than $m$ rotators, at least one of which is a diagonal rotator. At this point one or more eigenvalues can be found. This quadratic convergence implies that generally there are $\mathcal{O}(n)$ steps taken.</p>

<h2>The AMRVW decomposition of the companion matrix</h2>

<p>The main result of the two papers on "Fast and Backward Stable Computation of Roots of Polynomials" is a proof of backward stability of a method that utilizes a sparse factorization of both the <code>Q</code> and <code>R</code> parts of the QR decomposition of a companion matrix.</p>

<p>Returning to the real case, and digging into some structures, we can illustrate:</p>

In [1]:
ps = [24.0, -50.0, 35.0, -10.0, 1.0]
state = A.amrvw(ps)
state.QF.Q

AMRVW.DescendingChain{Float64,Float64,Array{AMRVW.Rotator{Float64,Float64},1}}(AMRVW.Rotator{Float64,Float64}[AMRVW.Rotator{Float64,Float64}(0.0, 1.0, 1), AMRVW.Rotator{Float64,Float64}(0.0, 1.0, 2), AMRVW.Rotator{Float64,Float64}(0.0, 1.0, 3)])

<p>To explain, this is a "chain" of real rotators, more clearly seen with:</p>

In [1]:
Vector(state.QF.Q)

3-element Array{AMRVW.Rotator{Float64,Float64},1}:
 AMRVW.Rotator{Float64,Float64}(0.0, 1.0, 1)
 AMRVW.Rotator{Float64,Float64}(0.0, 1.0, 2)
 AMRVW.Rotator{Float64,Float64}(0.0, 1.0, 3)

<p>A rotator is a matrix which is identical to the identity matrix except in the <code>&#91;i,i&#43;1&#93; × &#91;i, i&#43;1&#93;</code> block, in which case it takes the form of a rotator: <code>&#91;c s; -s c&#93;</code>. (Our rotators are in the different direction than those in the papers.) Here <code>c</code> and <code>s</code> are the cosine and sine of some angle. These rotators are indexed by <code>i</code> and we use the notation $U_i$ to indicate a rotator of this form for a given $i$. In the above, we  can see  with inspection that there are 3 rotators with $i$ being 1, 2, and 3. This set of rotators is "descending" due to their order (1 then 2 then 3); ascending would be 3 then 2 then 1. The product of descending rotators will be upper Hessenberg:</p>

In [1]:
Matrix(state.QF.Q)

4×4 Array{Float64,2}:
  0.0   0.0   0.0  1.0
 -1.0   0.0   0.0  0.0
  0.0  -1.0   0.0  0.0
  0.0   0.0  -1.0  0.0

<p>A rotator at level $i$ will commute with a rotator at level $j$ unless $|i-j| \leq 1$. In the case where $i-j = \pm 1$, a key computation is the "turnover", which represents $U_i V_j W_i$ as $VV_j WW_i UU_j$. With the turnover, we can easily pass a rotator through an ascending or descending chain without disturbing those patterns.</p>

<p>In the  above illustration of Francis's  algorithm, the matrices  $V_0$, $V_1$,  etc. can be seen to be  rotators of this type. More generally, a unitary matrix with $m$ shifts can be viewed as a product of $m$  such rotators.</p>

<p>The $R$ decomposition  is trickier.  In the initial QR decomposition, $R$ has a simple structure plus a rank one part (coming from the coefficients). The  decomposition has two chains, an ascending one, <code>Ct</code>, and a descending one, <code>B</code>:</p>

In [1]:
Ct = state.RF.Ct
B = state.RF.B

AMRVW.DescendingChain{Float64,Float64,Array{AMRVW.Rotator{Float64,Float64},1}}(AMRVW.Rotator{Float64,Float64}[AMRVW.Rotator{Float64,Float64}(-0.7536071065605803, 0.6573251318346122, 1), AMRVW.Rotator{Float64,Float64}(-0.8025327939615967, 0.5966080075025758, 2), AMRVW.Rotator{Float64,Float64}(-0.3843312210120439, 0.9231952732523013, 3), AMRVW.Rotator{Float64,Float64}(-0.04163054471218133, -0.999133073092352, 4)])

<p>These almost begin as inverses:</p>

In [1]:
M = diagm(0 => ones(5))
(Z = Ct * (B * M)) |> round2

5×5 Array{Float64,2}:
  1.0   0.0   0.0  0.0   0.0
 -0.0   1.0   0.0  0.0   0.0
 -0.0  -0.0   1.0  0.0   0.0
  0.0   0.0  -0.0  0.0  -1.0
  0.0   0.0   0.0  1.0   0.0

<p>However, <code>Ct</code> is cleverly chosen to encode the rank 1 part. This can be uncovered through the following:</p>

In [1]:
e1 = vcat(1, zeros(4))
en1 = vcat(zeros(4), 1)
rho = (en1' * (Ct * M) * e1)

-0.015072142131211606

<p>and</p>

In [1]:
yt = -(1/rho * en1' * (Ct*(B*M)))
Ct * (e1 * yt) |> round2

5×5 Array{Float64,2}:
  0.0   0.0  -0.0  -50.0   0.0
  0.0   0.0  -0.0  -35.0   0.0
  0.0   0.0  -0.0  -10.0   0.0
  0.0   0.0   0.0   24.0   0.0
 -0.0  -0.0  -0.0   -1.0  -0.0

<p>Leading to <code>R</code>:</p>

In [1]:
Z + Ct * (e1 * yt) |> round2

5×5 Array{Float64,2}:
 1.0  0.0  -0.0  -50.0   0.0
 0.0  1.0  -0.0  -35.0   0.0
 0.0  0.0   1.0  -10.0   0.0
 0.0  0.0   0.0   24.0  -1.0
 0.0  0.0   0.0    0.0   0.0

<p>The algorithm passes a rotator through this decomposition, which in turn relies on passing a rotator through the two chains <code>B</code> and <code>Ct</code>, which, with the turnover computation, is easily computed.</p>

<p>With these decompositions in mind, the computation above $V_0' F V_0$, can be seen as $U_1' Q R U_1$.  The product $U_1' Q = U_1' Q_1 Q_2 \cdots Q_k$ is just a product of two rotators at level 1, so $U_1' Q_1$ can be fused to give a new $\tilde{Q}_1$ in the descending chain factorization of $Q$.  The passthrough just mentioned allows $\tilde{Q} R U_1$ to have this form $\tilde{Q} \tilde{U}_1 \tilde{R}$ and by passing through the descending chain, we have this form $U_2 \hat{Q} \tilde{R}$. The matrix $V_1$ (of Francis's algorithm above) is seen to be $U_2$, as the similarity transform using $Q_1=U_2$ leaves the product $\hat{Q} \tilde{R} U_2$ having the same eigen values, but with the bulge shifted down one level. This basic idea forms the algorithm to chase the bulge. In the $m > 1$ case, some other details are included.</p>

<p>The decomposition of the companion matrix is sparse. Rather than require $O(n^2)$ storage, it only needs $O(n)$. The iterative algorithm is $O(n)$ per iteration  and  $O(n^2)$ overall, as compared to the $O(n^3)$ required in general. This reduction allows the method to be practical for large $n$, unlike <code>Polynomial.roots</code> which uses an $O(n^3)$ algorithm.</p>

<h3>Pencil decompositions</h3>

<p>In "<em>Fast and backward stable computation of roots of polynomials, Part II</em>" the method is extended to the pencil decomposition of a polynomial.  A pencil decomposition of a polynomial, is a specification where if $p = a_0 + a_1x^1 + \cdots + a_n x^n$ then $v_1 = a_0$, $v_{i+1} + w_i = a_i$, and $w_n = a_n$. This has some advantages in cases where the polynomial has a particularly small leading coefficient, since division by a tiny $a_n$ will result in very large entries. The algorithm uses two upper triangular matrices.</p>

<p>The <code>roots</code> function allows a pencil decomposition to be passed in as two vectors:</p>

In [1]:
ps = [24.0, -50.0, 35.0, -10.0, 1.0]
vs, ws = A.basic_pencil(ps)
A.roots(vs, ws)

4-element Array{Complex{Float64},1}:
 0.9999999999999978 + 0.0im
  2.000000000000016 + 0.0im
  2.999999999999957 + 0.0im
  4.000000000000038 + 0.0im

<h4>The Wilkinson polynomial</h4>

<p>The Wilkinson polynomial, $(x-1)(x-2)\cdots(x-20)$, poses a challenge for <code>roots</code>:</p>

In [1]:
import Polynomials
ps = Polynomials.coeffs(Polynomials.poly(1.0:20.0))
A.roots(ps)

20-element Array{Complex{Float64},1}:
  0.999999999999998 + 0.0im                 
 2.0000000000658806 + 0.0im                 
 2.9999999840979017 + 0.0im                 
  4.000000974327189 + 0.0im                 
   4.99997599385403 + 0.0im                 
  6.000307515747747 + 0.0im                 
  6.997597874719522 + 0.0im                 
  8.013488311881199 + 0.0im                 
  8.947674655127813 + 0.0im                 
 10.392884548291114 - 0.046155826720116366im
 10.392884548291114 + 0.046155826720116366im
 12.333430486570474 - 0.7766219381420817im  
 12.333430486570474 + 0.7766219381420817im  
 14.471183155493314 - 1.064208878232851im   
 14.471183155493314 + 1.064208878232851im   
 16.674979944732755 - 0.8714271268945346im  
 16.674979944732755 + 0.8714271268945346im  
 18.540903438336436 + 0.0im                 
 18.740138141986648 + 0.0im                 
 20.014956839679545 + 0.0im                 

<p>The answer involves complex-valued roots, even though the roots are clearly integer valued. (The <code>Polynomials.roots</code> function will also show this, though <code>PolynomialRoots.roots</code> will not. As an aside, the exact implementation of the fundamental <code>turnover</code> operation will effect the number of such roots.  The issue of spurious complex-valued roots might be addressed by separating out the smaller coefficients from the large ones. Here is a function to split a polynomial into two pieces.</p>

In [1]:
function pencil_split(ps, n)
    vs = zeros(eltype(ps), length(ps)-1)
    ws = zeros(eltype(ps), length(ps)-1)

    vs[1:n] = ps[1:n]
    ws[n:end] = ps[n+1:end]

    vs, ws
end

pencil_split (generic function with 1 method)

<p>It turns out that with <code>n&#61;13</code> only real roots are identified:</p>

In [1]:
A.roots(pencil_split(ps, 13)...)

20-element Array{Complex{Float64},1}:
 0.9999999999998948 + 0.0im
 2.0000000000486082 + 0.0im
 2.9999999937180775 + 0.0im
  4.000000300143694 + 0.0im
  4.999992866358705 + 0.0im
  6.000097021418143 + 0.0im
 6.9991960279584875 + 0.0im
   8.00425633381146 + 0.0im
   8.98593816770586 + 0.0im
  10.03059300220452 + 0.0im
 10.964284537318703 + 0.0im
  12.01023120254452 + 0.0im
 13.020869369796813 + 0.0im
 14.009268935347217 + 0.0im
 14.898793643212594 + 0.0im
  16.21570915641559 + 0.0im
  16.79892625882952 + 0.0im
 18.087132356819435 + 0.0im
 18.971255873673417 + 0.0im
 20.003454952675014 + 0.0im

<p>The pencil here does a better job with this polynomial, but the choice of <code>13</code> was made with hindsight, not foresight.</p>

<h2>Other uses</h2>

<p>The <code>AMRVW.jl</code> package allows some other applications, though the exact interface needs to be settled on.</p>

<p>If we take rotators $Q_1, Q_2, \dots, Q_k$ their product will be upper Hessenberg:</p>

In [1]:
Qs = A.random_rotator.(Float64, [1,2,3,4])

4-element Array{AMRVW.Rotator{Float64,Float64},1}:
 AMRVW.Rotator{Float64,Float64}(0.9934643274630929, 0.11414302457138886, 1)
 AMRVW.Rotator{Float64,Float64}(0.826551083941464, 0.5628617109336811, 2)  
 AMRVW.Rotator{Float64,Float64}(0.9318889738554799, 0.36274362903651497, 3)
 AMRVW.Rotator{Float64,Float64}(0.8443202221752373, 0.535838933286821, 4)  

In [1]:
M = diagm(0 => ones(5))
(F = Qs * M) |> round2

5×5 Array{Float64,2}:
  0.99   0.09   0.06   0.02  0.01
 -0.11   0.82   0.52   0.17  0.11
  0.0   -0.56   0.77   0.25  0.16
  0.0    0.0   -0.36   0.79  0.5 
  0.0    0.0    0.0   -0.54  0.84

<p>Their eigenvalues can be found:</p>

In [1]:
eigvals(F)

5-element Array{Complex{Float64},1}:
 0.6989812153008731 - 0.7151400287052276im 
 0.6989812153008731 + 0.7151400287052276im 
 0.9090188413300189 - 0.41675501929434516im
 0.9090188413300189 + 0.41675501929434516im
 0.9999999999999996 + 0.0im                

<p>But the sparse representation can be used to also find such eigenvalues:</p>

In [1]:
QF = A.q_factorization(A.DescendingChain(Qs))
state = A.QRFactorization(QF)  # defaults to identify R factorization
Matrix(state) |> round2 # same as F

5×5 Array{Float64,2}:
  0.99   0.09   0.06   0.02  0.01
 -0.11   0.82   0.52   0.17  0.11
  0.0   -0.56   0.77   0.25  0.16
  0.0    0.0   -0.36   0.79  0.5 
  0.0    0.0    0.0   -0.54  0.84

In [1]:
eigvals(state)

5-element Array{Complex{Float64},1}:
 0.6989812153008732 - 0.7151400287052279im 
 0.6989812153008732 + 0.7151400287052279im 
 0.9090188413300189 - 0.41675501929434544im
 0.9090188413300189 + 0.41675501929434544im
                1.0 + 0.0im                

<hr />

<p>Not all upper Hessenberg matrices can he expressed as a descending chain of rotators, as the latter is unitary. However, any upper Hessenberg matrix can easily be seen to be represented as a descending chain of rotators times an upper triangular matrix.</p>

<p>The Givens rotation is a rotator, $U$, chosen so that if $x= [a,b]$, then $Ux = [r,0]$. This allows, for example, the following:</p>

In [1]:
F = triu(rand(5,5), -1)  # upper Hessenberg

5×5 Array{Float64,2}:
 0.0662962  0.206716  0.201386  0.0732278  0.521177
 0.179678   0.532981  0.435744  0.225593   0.626793
 0.0        0.924176  0.219759  0.140399   0.836273
 0.0        0.0       0.817677  0.292295   0.800583
 0.0        0.0       0.0       0.50208    0.392074

In [1]:
c,s,r = A.givensrot(F[1,1], F[2,1])
U1 = A.Rotator(c,s,1)
U1 * F |> round2

5×5 Array{Float64,2}:
 0.19   0.57   0.48  0.24   0.77
 0.0   -0.01  -0.04  0.01  -0.27
 0.0    0.92   0.22  0.14   0.84
 0.0    0.0    0.82  0.29   0.8 
 0.0    0.0    0.0   0.5    0.39

<p>That is the subdiagonal in column 1 is 0 Similarly, a <code>U2</code> could then be found so that subdiagonal in column 2 is 0, etc. That is a choice of rotators is available for $U_k U_{k-1} U_{k-2} \cdots U_2 U_1 F = R$. Setting $V_i = U_i'$, we have then $F = V_1 V_2 \cdots V_k R = QR$, where $Q$ is unitary and  in decomposed form, and $R$ is upper triangular.</p>

<p>For example:</p>

In [1]:
Us = Any[]
G = copy(F)
for i in 1:4
  c,s,r = A.givensrot(G[i,i], G[i+1,i])
  Ui =  A.Rotator(c,s,i)
  pushfirst!(Us, Ui)
  mul!(Ui, G)
end
R = G
Qs = reverse(adjoint.(Us))

4-element Array{AMRVW.Rotator{Float64,Float64},1}:
 AMRVW.Rotator{Float64,Float64}(0.34616051204472437, -0.938175303395338, 1)  
 AMRVW.Rotator{Float64,Float64}(-0.01021293100303493, -0.9999478466601781, 2)
 AMRVW.Rotator{Float64,Float64}(0.043803958532509554, -0.999040145948541, 3) 
 AMRVW.Rotator{Float64,Float64}(0.046987602226890364, -0.9988954726281263, 4)

<p>With this, we can do the following:</p>

In [1]:
QF = A.q_factorization(A.DescendingChain(Qs))
RF = A.RFactorizationUpperTriangular(R)

state = A.QRFactorization(QF, RF)

Matrix(state)  - F |> round2# same as F

5×5 Array{Float64,2}:
 0.0   0.0   0.0  -0.0   0.0
 0.0   0.0   0.0   0.0   0.0
 0.0  -0.0  -0.0  -0.0  -0.0
 0.0   0.0  -0.0  -0.0  -0.0
 0.0   0.0   0.0   0.0   0.0

<p>And</p>

In [1]:
[eigvals(state) eigvals(F)]

5×2 Array{Complex{Float64},2}:
 -0.278281-0.311004im   -0.278281-0.311004im 
 -0.278281+0.311004im   -0.278281+0.311004im 
  0.255447-0.0349049im   0.255447-0.0349049im
  0.255447+0.0349049im   0.255447+0.0349049im
   1.54907+0.0im          1.54907+0.0im      

<p>With this patttern, we might provide an <code>eigvals</code> for <code>Hessenberg</code> matrices:</p>

In [1]:
function LinearAlgebra.eigvals(H::Hessenberg)
  R = Matrix(H.H) # need this for R
  S = eltype(R)
  T = real(S)
  N = size(R)[1]
  Qs = Vector{A.Rotator{T,S}}(undef, N-1)
  for i in 1:N-1
    c,s,r = A.givensrot(R[i,i], R[i+1,i])
    Ui =  A.Rotator(c,s,i)
    Qs[i] = Ui'
    mul!(Ui, R)
  end

  QF = A.q_factorization(A.DescendingChain(Qs))
  RF = A.RFactorizationUpperTriangular(R)

  state = A.QRFactorization(QF, RF)

  eigvals(state)

end

In [1]:
H = hessenberg(rand(5,5))
[eigvals(H) eigvals(Matrix(H.H))]

5×2 Array{Complex{Float64},2}:
  -0.48111+0.0im        -0.48111+0.0im     
 0.0398371-0.298497im  0.0398371-0.298497im
 0.0398371+0.298497im  0.0398371+0.298497im
  0.444337+0.0im        0.444337+0.0im     
   2.67512+0.0im         2.67512+0.0im     

<p>Sadly this is not competitive performance-wise with <code>eigvals&#40;Matrix&#40;H.H&#41;&#41;</code>, as the <code>R</code> part is not factored. Though, this approach can be made to work with other floating point types, such as <code>BigFloat</code>, that <code>LinearAlgebra.eigvals</code> can not currently handle.</p>

<hr />

<p>The product of a descending chain of rotators is upper Hessenberg and the product of an  ascending chain or rotators is lower Hessenberg. With modifications, described in "A generalization of the multishift QR algorithm" a related algorithm can be employed. A "twisted chain" is one where a descending chain of rotators is permuted, an ascending chain being one possible twisting among many others.</p>

In [1]:
N = 4
T = Float64; S = T # real case here
M = diagm(0 => ones(N+1))
Qs = A.random_rotator.(T, [4,3,2,1])
F = Qs * M

QF = A.q_factorization(A.TwistedChain(Qs))
state = A.QRFactorization(QF)   # use default identify R factorization

[eigvals(state) eigvals(F)]

5×2 Array{Complex{Float64},2}:
 -0.132935-0.991125im  -0.132935-0.991125im
 -0.132935+0.991125im  -0.132935+0.991125im
  0.805115-0.593119im   0.805115-0.593119im
  0.805115+0.593119im   0.805115+0.593119im
       1.0+0.0im             1.0+0.0im     

<p>Here is another example with a CMV pattern to the twisted</p>

In [1]:
N = 5
M = diagm(0 => ones(N+1))
Qs = A.random_rotator.(T, [1,3,5,2,4])
F = Qs * M

QF = A.q_factorization(A.TwistedChain(Qs))

state = A.QRFactorization(QF)

[eigvals(state) eigvals(F)]

6×2 Array{Complex{Float64},2}:
 0.00579598-0.999983im  0.00579598-0.999983im
 0.00579598+0.999983im  0.00579598+0.999983im
    0.76368-0.645595im     0.76368-0.645595im
    0.76368+0.645595im     0.76368+0.645595im
   0.964397-0.264458im    0.964397-0.264458im
   0.964397+0.264458im    0.964397+0.264458im

<p>The implementation for twisted chains is not nearly as efficient as that for descending chains.</p>